# Ex. 1a

$$
\hat{\theta} = \left(\tilde X^T\cdot \tilde X\right)^{-1}\cdot\tilde X^T\cdot Y\text{.}
$$

In [1]:
#!/usr/bin/env python3

import numpy as np
import pytest
from sklearn.linear_model import LinearRegression

class LinearRegr:
    
    def fit(self, X, Y):
        # input:
        #  X = np.array, shape = (n, m)
        #  Y = np.array, shape = (n)
        # Finds theta minimising quadratic loss function L, using an explicit formula.
        # Note: before applying the formula to X one should append to X a column with ones.
        n, m = X.shape
        X = np.c_[np.ones(n), X] #r_ would stack on the bottom
        temp_mat = X.T @ X
        if np.linalg.det(temp_mat) == 0:
            raise ValueError('Not able to inverse matrix')
        self.theta = (np.linalg.inv(temp_mat) @ X.T) @ Y #det cannot be 0
        return self
    
    def predict(self, X):
        # input:
        #  X = np.array, shape = (k, m)
        # returns:
        #  Y = wektor(f(X_1), ..., f(X_k))
        k, m = X.shape
        X = np.c_[np.ones(k), X]
        Y = X @ self.theta
        return Y


def test_RegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    a = np.array([1,2,10]).reshape((3,1))
    expected = LinearRegression().fit(X, Y).predict(a)
    actual = LinearRegr().fit(X, Y).predict(a)
    print(f'actual: {actual}')
    print(f'expected: {expected}')
    assert list(actual) == pytest.approx(list(expected))

def test_RegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    a = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    expected = LinearRegression().fit(X, Y).predict(a)
    actual = LinearRegr().fit(X, Y).predict(a)
    print(f'actual: {actual}')
    print(f'expected: {expected}')
    assert list(actual) == pytest.approx(list(expected))
    


In [2]:
test_RegressionInOneDim()

actual: [ 1.8         3.34285714 15.68571429]
expected: [ 1.8         3.34285714 15.68571429]


In [3]:
test_RegressionInThreeDim()

actual: [ 2.25 -1.75  2.75  2.    3.75]
expected: [ 2.25 -1.75  2.75  2.    3.75]


# Ex. 1b

Ridge regresssion:

$$
\tilde Y = \tilde X^T\cdot \theta,
$$

Gradient:

$$
\frac{\partial L}{\partial\theta_0} = -2\sum\limits_{i=1}^n\left(y_i-\theta_0-\theta_1x_{i1}-\dots-\theta_p x_{ip}\right)
=-2(Y-X\cdot\theta)
$$

$$
\frac{\partial L}{\partial\theta_j} =-2\sum\limits_{i=1}^nx_{ij}\left(y_i-\theta_0-\theta_1x_{i1}-\dots-\theta_p x_{ip}\right)+2\alpha\theta_j
$$

$$
\begin{align}
\begin{bmatrix}
\left(\frac{\partial L}{\partial\theta_0}\right)^*\\
\frac{\partial L}{\partial\theta_1}\\
\vdots\\
\frac{\partial L}{\partial\theta_p}
\end{bmatrix} & =
-2\cdot
\begin{bmatrix}
\sum\limits_{i=1}^nx_{i0}\left(y_i-\theta_0-\theta_1x_{i1}-\dots-\theta_p x_{ip}\right)\\
\sum\limits_{i=1}^nx_{i1}\left(y_i-\theta_0-\theta_1x_{i1}-\dots-\theta_p x_{ip}\right)\\
\sum\limits_{i=1}^nx_{i2}\left(y_i-\theta_0-\theta_1x_{i1}-\dots-\theta_p x_{ip}\right)\\
\vdots\\
\sum\limits_{i=1}^nx_{ip}\left(y_i-\theta_0-\theta_1x_{i1}-\dots-\theta_p x_{ip}\right)\\
\end{bmatrix}+ 2\alpha\cdot
\begin{bmatrix}
\theta_0\\
\theta_1\\
\vdots\\
\theta_p
\end{bmatrix}\\ & =
-2\cdot
\begin{bmatrix}
1 & 1 & \cdots & 1\\
x_{11} & x_{21} & \cdots & x_{n1}\\
x_{12} & x_{22} & \cdots & x_{n2}\\
\vdots & \vdots & \cdots & \vdots\\
x_{1p} & x_{2p} & \cdots & x_{np}\\
\end{bmatrix}\cdot
\begin{bmatrix}
y_1 - \theta_0 - \theta_1 x_{11} - \theta_2 x_{12} - \cdots - \theta_p x_{1p}\\
y_2 - \theta_0 - \theta_1 x_{21} - \theta_2 x_{22} - \cdots - \theta_p x_{2p}\\
\vdots \\
y_n - \theta_0 - \theta_1 x_{n1} - \theta_2 x_{n2} - \cdots - \theta_p x_{np}
\end{bmatrix}+ 2\alpha\cdot
\begin{bmatrix}
\theta_0\\
\theta_1\\
\vdots\\
\theta_p
\end{bmatrix}\\ & =
-2\cdot
\begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1p}\\
1 & x_{21} & x_{22} & \cdots & x_{2p}\\
\vdots & \vdots & \cdots & \vdots\\
1 & x_{n1} & x_{n2} & \cdots & x_{np}
\end{bmatrix}^T\cdot
\begin{bmatrix}
y_1 - \theta_0 - \theta_1 x_{11} - \theta_2 x_{12} - \cdots - \theta_p x_{1p}\\
y_2 - \theta_0 - \theta_1 x_{21} - \theta_2 x_{22} - \cdots - \theta_p x_{2p}\\
\vdots \\
y_n - \theta_0 - \theta_1 x_{n1} - \theta_2 x_{n2} - \cdots - \theta_p x_{np}
\end{bmatrix}+ 2\alpha\cdot
\begin{bmatrix}
\theta_0\\
\theta_1\\
\vdots\\
\theta_p
\end{bmatrix}\\ & =
-2\cdot
\begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1p}\\
1 & x_{21} & x_{22} & \cdots & x_{2p}\\
\vdots & \vdots & \cdots & \vdots\\
1 & x_{n1} & x_{n2} & \cdots & x_{np}
\end{bmatrix}^T\cdot
\left(
\begin{bmatrix}
y_1\\
y_2\\
\vdots\\
y_n
\end{bmatrix}-
\begin{bmatrix}
\theta_0 + \theta_1 x_{11} + \theta_2 x_{12} + \cdots + \theta_p x_{1p}\\
\theta_0 + \theta_1 x_{21} + \theta_2 x_{22} + \cdots + \theta_p x_{2p}\\
\vdots \\
\theta_0 + \theta_1 x_{n1} + \theta_2 x_{n2} + \cdots + \theta_p x_{np}
\end{bmatrix}\right) + 2\alpha\cdot
\begin{bmatrix}
\theta_0\\
\theta_1\\
\vdots\\
\theta_p
\end{bmatrix}\\
& =-2\cdot
\begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1p}\\
1 & x_{21} & x_{22} & \cdots & x_{2p}\\
\vdots & \vdots & \cdots & \vdots\\
1 & x_{n1} & x_{n2} & \cdots & x_{np}
\end{bmatrix}^T\cdot
\left(
\begin{bmatrix}
y_1\\
y_2\\
\vdots\\
y_n
\end{bmatrix}-
\begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1p}\\
1 & x_{21} & x_{22} & \cdots & x_{2p}\\
\vdots & \vdots & \vdots & \cdots & \vdots\\
1 & x_{n1} & x_{n2} & \cdots & x_{np}
\end{bmatrix}\cdot
\begin{bmatrix}
\theta_0\\
\theta_1\\
\vdots\\
\theta_p
\end{bmatrix}\right) + 2\alpha\cdot
\begin{bmatrix}
\theta_0\\
\theta_1\\
\vdots\\
\theta_p
\end{bmatrix}\\
& =-2\cdot X^T\cdot(Y-X\cdot\theta) + 2\alpha\theta
\end{align}
$$

\* here it's calculated the same way as for the other $j$ but then is substituted with the correct formula.

For gradient:
$$
\begin{align}
& -2\cdot (Y-X\cdot\theta)\\
& -2\cdot X^T\cdot(Y-X\cdot\theta) + 2\alpha\theta
\end{align}
$$

$$
\theta_{n+1} = \theta_{n} - c\nabla_\theta L
$$

* $c$ - learning rate
* $L$ - cost function

In [19]:
#!/usr/bin/env python3

import numpy as np
import pytest
from sklearn.linear_model import Ridge

class RidgeRegr:
    def __init__(self, alpha = 0.0):
        self.alpha = alpha

    def fit(self, X, Y):
        # input:
        #  X = np.array, shape = (n, m)
        #  Y = np.array, shape = (n)
        # Finds theta (approximately) minimising quadratic loss function L with Ridge penalty,
        # using an iterative method.
        n, m = X.shape
        X = np.c_[np.ones(n), X]
        
        theta = np.zeros(m+1)
        c = 5*10**(-3)
        iter_count = 5*10**4
        for i in range(iter_count):
            L_grad = -2*(X.T @ (Y-X @ theta)) + 2 * self.alpha * theta
            L_grad[0] = -2*sum(Y-X @ theta)
            theta = theta - c * L_grad
        self.theta = theta
        
        return self
    
    def predict(self, X):
        # input:
        #  X = np.array, shape = (k, m)
        # returns:
        #  Y = wektor(f(X_1), ..., f(X_k))
        k, m = X.shape
        X = np.c_[np.ones(k), X]
        Y = X @ self.theta
        return Y


def test_RidgeRegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,2,10]).reshape((3,1))
    alpha = 0.3
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    print(f'actual: {actual}')
    print(f'expected: {expected}')
    assert list(actual) == pytest.approx(list(expected), rel=1e-5)

def test_RidgeRegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    alpha = 0.4
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    print(f'actual: {actual}')
    print(f'expected: {expected}')
    assert list(actual) == pytest.approx(list(expected), rel=1e-3)
    


In [20]:
test_RidgeRegressionInOneDim()

actual: [ 1.88950276  3.38121547 15.31491713]
expected: [ 1.88950276  3.38121547 15.31491713]


In [21]:
test_RidgeRegressionInThreeDim()

actual: [ 0.54685378 -1.76188321  1.58691716  5.15527388  3.66704391]
expected: [ 0.54685378 -1.76188321  1.58691716  5.15527388  3.66704391]
